In [80]:
from funcs import *
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import sys

## Sample Data to Test Nueral Network

In [81]:
X, y = load_boston(return_X_y=True)

scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [174]:
x = np.array([[1, 2], [3, 4]])
x

array([[1, 2],
       [3, 4]])

array([[ 1,  4],
       [ 9, 16]])

In [177]:
x_sq = x ** 2
x_sq
np.sum(x_sq)

30

## Neural Network Class

In [215]:
class NeuralNetwork:
    
    def __init__(self, layers=None, nodes=None, nnodes=None, 
                 activations=[], activationFn="relu", batchSize=50, 
                 lr=.001, lr_type="constant", power_t=.5,
                 annealing_rate=.999, max_epoch=200, momentum=.9, 
                 tol=0.0001, alpha=.0001, shuffle=False, 
                 early_stopping=False, num_epochs_stop=50):
        
        if layers != None:
            self.layers = layers # total number of hidden layers
        else:
            self.layers = len(nodes)

        # an int array of size [0, ..., Layers + 1]
        # Nodes[0] shall represent the input size (typically 50)
        # Nodes[Layers + 1] shall represent the output size (typically 1)
        # all other Nodes represent the number of nodes (or width) in the hidden layer i
        self.nodes = nodes
        if nodes != None:
            self.nodes.insert(0, batchSize)
            self.nodes.append(1)
        
        # alternative to nodes where each hidden layer of the nueral network is the same size
        self.nnodes = nnodes
        if nnodes != None:
            self.nodes = []
            self.nodes.append(batchSize)
            for i in range(layers):
                self.nodes.append(nnodes)
            self.nodes.append(1)
        
        # activations[i] values are labels indicating the activation function used in layer i
        self.activations = activations
        self.activationFn = activationFn
        if activationFn != "":
            self.activations = [activationFn] * self.layers
        
        self.batchSize = batchSize
        self.lr = lr
        self.lr_type = lr_type
        self.power_t = power_t
        self.annealing_rate = annealing_rate
        self.max_epoch = max_epoch
        self.mu = momentum
        self.tol = tol
        self.alpha = alpha
        self.shuffle = shuffle
        
        if early_stopping == False:
            self.num_epochs_stop = max_epoch
        else:
            self.num_epochs_stop = num_epochs_stop
    
        self.layer_values = [None] * (self.layers + 2)
        self.iters = 0
        self.epochs = 0
                
    def validateHyperParams(self):
        
        if self.layers != (len(self.nodes) - 2):
            raise ValueError("layers must be equal to the number of hidden layers, got %s." % self.layers)
        if self.nnodes != None and self.nnodes <= 0:
            raise ValueError("nnodes must be > 0, got %s." % self.nnodes)
        if self.lr <= 0 or self.lr > 1:
            raise ValueError("lr must be in (0, 1], got %s." % self.lr)
            
        if self.lr_type not in ["constant", "invscaling", "annealing", "adaptive"]:
            raise ValueError("lr_type is not valid" % self.lr_type
                            + "\nAvailable lr types: constant, invscaling, adaptive")
            
        if self.max_epoch <= 0:
            raise ValueError("max_iter must be > 0, got %s." % self.max_epoch)
               
        activation_functions = list(ACTIVATIONS.keys())
        if self.activationFn != "":
            if self.activationFn not in activation_functions:
                raise ValueError("%s is not an activation function" % self.activationFn
                                + "\nAvailable activation functions: relu, leaky_relu, sigmoid, tanh")
    
    def initialize_weights(self, M):
        weights = []
        
        for i in range(self.layers + 1):
            if i == 0:
                input_size = M # special case for w1
            else:
                input_size = self.nodes[i]
            output_size = self.nodes[i + 1]
            
            # Xavier (Glorot) Initialization
            if self.activationFn == "tanh":
                target_variance = 2 / (input_size + output_size)
                w_i = np.random.normal(loc= 0, scale = np.sqrt(target_variance), size=(input_size, output_size))
            # He Initialization
            elif self.activationFn == "relu":
                target_variance = 2 / input_size
                w_i = np.random.normal(loc= 0, scale = np.sqrt(target_variance), size=(input_size, output_size))
            # Random Uniform
            else:
                w_i = np.random.uniform(-1/np.sqrt(input_size), 1/np.sqrt(input_size))
                #w_i = np.random.normal(size=(input_size, output_size))
            w_i = np.round(w_i, 2)
            w_i[input_size - 1:] = 0 # initialize bias to 0
            weights.append(w_i)
        return weights
    
    # returns the weight term for L2 regularization
    def get_weight_term(self):
        weight_term = 0
        for i in range(len(self.weights)):
            weight_term = np.sum(self.weights[i] ** 2)
        return weight_term
        
    def forward_pass(self, X_batch, y_batch):
        
        self.layer_values[0] = X_batch
        
        # calculate hidden layers
        for i in range(self.layers):
            X = self.layer_values[i]
            weights = self.weights[i]
            h_layer = X.dot(weights)
            
            # apply activation function
            activation_fn = ACTIVATIONS[self.activations[i]]
            activation_fn(h_layer)
            self.layer_values[i + 1] = h_layer
            
        
        # calculate predictions
        X = self.layer_values[self.layers] # values in last hidden layer
        weights = self.weights[self.layers]
        y_pred = X.dot(weights)
        y_pred = y_pred.flatten()
        
        # calculate the l2 loss
        l2_loss = 0
        # only need predictions once we have fit the data
        if isinstance(y_batch, np.ndarray): 
            l2_loss = squared_loss(y_pred, y_batch) # l2
            weight_term = self.get_weight_term()
            l2_loss += self.alpha * weight_term # l2 regularization
            self.layer_values[self.layers + 1] = l2_loss
        
        return l2_loss, y_pred
    
    
    def backward_pass(self, y_pred, y_batch):
        
        # loss layer
        J = squared_loss_derivative(y_pred, y_batch, self.batchSize)
        J = np.reshape(J, (len(J), 1))
        
        J_weights = [None] * (self.layers + 1)
        
        # output layer
        # jacobian w.r.t. weights
        x_t = self.layer_values[self.layers].T
        J_wi = x_t.dot(J)
        J_weights[self.layers] = J_wi
        
        # update jacobian at output layer
        w_t = self.weights[self.layers].T
        w_t = np.delete(w_t, w_t.shape[1] - 1, 1) # take out the bias
        J = np.dot(J, w_t)
        zeros = [0] * len(J)
        zeros = np.reshape(zeros, (len(J), 1))
        J = np.append(J, zeros, axis=1)
        
        # iterate through hidden layers backwards
        for i in range(self.layers, 0 , -1):
            # update jacobian at activation layer
            d_activation_fn = DERIVATIVES[self.activations[i - 1]]
            d_activation_fn(self.layer_values[i], J)
            
            # hidden layer
            # jacobian w.r.t. weights
            x_t = self.layer_values[i - 1].T
            J_wi = x_t.dot(J)
            J_weights[i - 1] = J_wi
            
            # jacobian w.r.t. inputs
            w_t = self.weights[i - 1].T
            w_t = np.delete(w_t, w_t.shape[1] - 1, 1)
            J = np.dot(J, w_t)
            zeros = [0] * len(J)
            zeros = np.reshape(zeros, (len(J), 1))
            J = np.append(J, zeros, axis=1)
            
            
        # initialize velocity to 0
        if self.epochs == 0 and self.iters == 0:
            self.velocity = []
            for i in range(len(J_weights)):
                n_rows = J_weights[i].shape[0]
                n_cols = J_weights[i].shape[1]
                vel_i = np.zeros((n_rows, n_cols))
                self.velocity.append(vel_i)
        
        for i in range(len(J_weights)):
            self.velocity[i] = self.mu * self.velocity[i] - self.lr * J_weights[i]
            self.weights[i] += self.velocity[i]
      
    
    def fit(self, X_train, y_train):
        
        self.validateHyperParams()
        # convert to numpy arrays
        if isinstance(X_train, pd.DataFrame):
            X_train = X_train.to_numpy()
            
        if isinstance(y_train, pd.Series):
            y_train = y_train.to_numpy()
            
        # add ones for bias
        ones = [1] * len(X_train)
        ones = np.reshape(ones, (len(X_train), 1))
        X_train = np.append(X_train, ones, axis=1)
        
        # save 10% for validation
        val_rows = round(len(X_train) * .1)
        X_val = X_train[:val_rows, :]
        y_val = y_train[:val_rows]
        
        X_train = X_train[val_rows:, :]
        y_train = y_train[val_rows:]
        
        # initalize weights on first iteration
        M = X_train.shape[1] # M = number of features
        self.weights = self.initialize_weights(M)
        
        best_v_loss = np.inf
        n_epoch_no_change = 0 
        while (self.epochs < self.max_epoch and n_epoch_no_change <= self.num_epochs_stop):
            # ONE EPOCH 
            last_idx = 0
            if self.shuffle == True: # shuffle data after every epoch, if specified 
                np.random.shuffle(X_train)
            while (last_idx < len(X_train)):
                first_idx = self.iters * self.batchSize
                remaining_rows = len(X_train) - first_idx
                last_idx = first_idx + min(self.batchSize, remaining_rows)
                X_batch = X_train[first_idx: last_idx, :]
                y_batch = y_train[first_idx: last_idx]

                loss, y_pred = self.forward_pass(X_batch, y_batch)
                self.backward_pass(y_pred, y_batch)
                self.iters += 1
            
            # trainig and validation loss after one epoch
            t_loss, y_pred = self.forward_pass(X_train, y_train)
            v_loss, y_pred = self.forward_pass(X_val, y_val)
            print("epoch:", self.epochs)
            print("training loss:", t_loss)
            print("validation loss:", v_loss)
            
            self.iters = 0 # start over, next epoch
            self.epochs += 1
            
            # decrease the learning rate by one of three methods, if specified
            if self.lr_type == "invscaling":
                self.lr = self.lr/pow(self.epochs, self.power_t)
            elif self.lr_type == "annealing":
                self.lr = self.lr * self.annealing_rate
            elif self.lr_type == "adaptive":
                if n_epoch_no_change >= 2: 
                    self.lr = self.lr/5
                
            # stops when validation loss doesn't improve for num_epochs_stop
            if best_v_loss - v_loss < self.tol: 
                n_epoch_no_change += 1
            else:
                n_epoch_no_change = 0
            # update best_v_loss
            if v_loss < best_v_loss:
                best_v_loss = v_loss
            
            
        
    def predict(self, X_test):
        
        # convert to numpy array
        if isinstance(X_test, pd.DataFrame):
            X_test = X_test.to_numpy()
        
        # add ones for bias
        ones = [1] * len(X_test)
        ones = np.reshape(ones, (len(X_test), 1))
        X_test = np.append(X_test, ones, axis=1)
        
        loss, y_pred = self.forward_pass(X_test, None)
        return y_pred
        

## Running Nueral Network on the Data

In [ ]:
nodes = [100, 50, 100] # use to specify a number of hidden nodes per layer
activations = [] # use if you want a diff activationFn per layer

nn = NeuralNetwork(layers=3, nnodes=100, batchSize=50, 
                   activationFn="tanh", lr=.001, lr_type="constant", 
                   max_epoch=2000, momentum=0.9, early_stopping=False)
nn.fit(X_std, y)

epoch: 0
training loss: 258.96249394151096
validation loss: 179.58621755267373
epoch: 1
training loss: 158.49421743843573
validation loss: 116.93733145476338
epoch: 2
training loss: 65.69780579409617
validation loss: 33.178636085198576
epoch: 3
training loss: 36.100177093504975
validation loss: 17.984436413962502
epoch: 4
training loss: 26.77662338261293
validation loss: 21.317553976781113
epoch: 5
training loss: 25.03125842963376
validation loss: 22.34995920865861
epoch: 6
training loss: 24.854632593841064
validation loss: 7.220059998571435
epoch: 7
training loss: 25.437037047808857
validation loss: 15.511320959313739
epoch: 8
training loss: 17.72269073318845
validation loss: 7.297130710859488
epoch: 9
training loss: 12.36527579117617
validation loss: 6.759317625314018
epoch: 10
training loss: 10.522415157017361
validation loss: 2.504496143144288
epoch: 11
training loss: 10.659100459490784
validation loss: 5.389246004547442
epoch: 12
training loss: 13.535812871665229
validation loss: 

epoch: 105
training loss: 3.162801253743333
validation loss: 6.45104525394246
epoch: 106
training loss: 3.2099505490888514
validation loss: 4.634431664532554
epoch: 107
training loss: 3.1115018527936025
validation loss: 6.453214046098934
epoch: 108
training loss: 3.0782749835114087
validation loss: 4.535663393601894
epoch: 109
training loss: 2.8901297080506185
validation loss: 6.166347157517038
epoch: 110
training loss: 2.8116969450004383
validation loss: 4.367649856263496
epoch: 111
training loss: 2.602331034016876
validation loss: 5.724474107870559
epoch: 112
training loss: 2.512408081441212
validation loss: 4.251242783782784
epoch: 113
training loss: 2.34348109883928
validation loss: 5.245894998356762
epoch: 114
training loss: 2.275859907094574
validation loss: 4.221642360899642
epoch: 115
training loss: 2.1616153539260354
validation loss: 4.84993510956824
epoch: 116
training loss: 2.1170470935580914
validation loss: 4.253891667768144
epoch: 117
training loss: 2.0514538690399773
val

epoch: 210
training loss: 1.905105650348697
validation loss: 6.031308919921277
epoch: 211
training loss: 2.1145541492143565
validation loss: 5.965204511927546
epoch: 212
training loss: 1.9504560281695804
validation loss: 6.052540312953463
epoch: 213
training loss: 1.9176568761697212
validation loss: 5.845859460287438
epoch: 214
training loss: 1.6125793655808296
validation loss: 5.624071122122277
epoch: 215
training loss: 1.445436557244531
validation loss: 5.504776566107526
epoch: 216
training loss: 1.3027677677527096
validation loss: 5.103099160715209
epoch: 217
training loss: 1.2898209732248151
validation loss: 5.480320396226983
epoch: 218
training loss: 1.4465165556809436
validation loss: 5.275756301659673
epoch: 219
training loss: 1.6052389333921107
validation loss: 5.766783958338969
epoch: 220
training loss: 1.835921111104104
validation loss: 5.726025065787502
epoch: 221
training loss: 1.8120410405874257
validation loss: 5.963041182733669
epoch: 222
training loss: 1.849412969827258

epoch: 315
training loss: 0.8598697366406598
validation loss: 6.033981498384876
epoch: 316
training loss: 0.8542596896152657
validation loss: 6.0006613293915505
epoch: 317
training loss: 0.8533564614852497
validation loss: 6.051803785133269
epoch: 318
training loss: 0.8473831058623703
validation loss: 6.025760050696484
epoch: 319
training loss: 0.8453121368778138
validation loss: 6.064738109320017
epoch: 320
training loss: 0.840913764671134
validation loss: 6.048257484606348
epoch: 321
training loss: 0.8381170546813442
validation loss: 6.081614046433825
epoch: 322
training loss: 0.8348467587629377
validation loss: 6.065515407422839
epoch: 323
training loss: 0.8317228751002668
validation loss: 6.10300495993467
epoch: 324
training loss: 0.8285818959671946
validation loss: 6.07735728016246
epoch: 325
training loss: 0.8257064059161854
validation loss: 6.126979099182169
epoch: 326
training loss: 0.8222953208990632
validation loss: 6.085100268905269
epoch: 327
training loss: 0.82000080689843

epoch: 420
training loss: 0.6385879194247289
validation loss: 5.90873312427484
epoch: 421
training loss: 0.6382347327811188
validation loss: 6.094236382542136
epoch: 422
training loss: 0.6336350962183401
validation loss: 5.926827612196892
epoch: 423
training loss: 0.6340915650951026
validation loss: 6.124876414549368
epoch: 424
training loss: 0.6299177066298308
validation loss: 5.932646906983209
epoch: 425
training loss: 0.6312163626244133
validation loss: 6.154971944794799
epoch: 426
training loss: 0.6273544725867621
validation loss: 5.932577691202224
epoch: 427
training loss: 0.6295991349335801
validation loss: 6.186827253653306
epoch: 428
training loss: 0.6265456086461004
validation loss: 5.922675966785823
epoch: 429
training loss: 0.6304149592923852
validation loss: 6.228574433587841
epoch: 430
training loss: 0.6283599313547508
validation loss: 5.900449338858287
epoch: 431
training loss: 0.6347671824795169
validation loss: 6.28083835032336
epoch: 432
training loss: 0.63415995252121

...

epoch: 1050
training loss: 0.16906187186984992
validation loss: 5.502541749468663
epoch: 1051
training loss: 0.16754998811505467
validation loss: 5.699041813294482
epoch: 1052
training loss: 0.16576075687864958
validation loss: 5.517028610626569
epoch: 1053
training loss: 0.16424878984410937
validation loss: 5.679477333384151
epoch: 1054
training loss: 0.16258414935465662
validation loss: 5.53394601397254
epoch: 1055
training loss: 0.16114221208444643
validation loss: 5.658070014204006
epoch: 1056
training loss: 0.15970810542976469
validation loss: 5.5533615012429856
epoch: 1057
training loss: 0.15842379470358825
validation loss: 5.63506420509557
epoch: 1058
training loss: 0.1573424887514124
validation loss: 5.575140619760916
epoch: 1059
training loss: 0.15631128558807955
validation loss: 5.610931054230473
epoch: 1060
training loss: 0.15570208030867588
validation loss: 5.598835570863429
epoch: 1061
training loss: 0.15500746046696318
validation loss: 5.586409481736497
epoch: 1062
traini

epoch: 1155
training loss: 0.1310385811982018
validation loss: 5.3553371304390165
epoch: 1156
training loss: 0.12695553000287682
validation loss: 5.484557023516166
epoch: 1157
training loss: 0.12457997297947326
validation loss: 5.416665221189416
epoch: 1158
training loss: 0.12323501438991778
validation loss: 5.407868557726934
epoch: 1159
training loss: 0.12517652692791723
validation loss: 5.497406047759851
epoch: 1160
training loss: 0.12834472761563626
validation loss: 5.33090019296642
epoch: 1161
training loss: 0.13562117839643223
validation loss: 5.588757605248134
epoch: 1162
training loss: 0.14297333643753585
validation loss: 5.26647073427719
epoch: 1163
training loss: 0.15355591378702912
validation loss: 5.6734624798093245
epoch: 1164
training loss: 0.16242334393376773
validation loss: 5.219921115380912
epoch: 1165
training loss: 0.17110189904898884
validation loss: 5.729139276647871
epoch: 1166
training loss: 0.17600897222392245
validation loss: 5.197045538256634
epoch: 1167
train

epoch: 1260
training loss: 0.6574086498921382
validation loss: 5.546497687363475
epoch: 1261
training loss: 0.3706746043197684
validation loss: 5.317189196334301
epoch: 1262
training loss: 0.8466956515598552
validation loss: 4.6923813487633845
epoch: 1263
training loss: 0.675653509159704
validation loss: 5.884443141622956
epoch: 1264
training loss: 0.23310470996146215
validation loss: 4.47496054305915
epoch: 1265
training loss: 0.7731278586568164
validation loss: 5.237147659203494
epoch: 1266
training loss: 1.1822902180664472
validation loss: 6.459893580725258
epoch: 1267
training loss: 0.5689910412606759
validation loss: 4.936868066528156
epoch: 1268
training loss: 0.18633754450754858
validation loss: 4.7949756018753895
epoch: 1269
training loss: 0.5006116139979637
validation loss: 5.365723058207181
epoch: 1270
training loss: 0.6732720113034751
validation loss: 4.910713888788172
epoch: 1271
training loss: 0.2866725094430984
validation loss: 5.330978226177993
epoch: 1272
training loss:

epoch: 1365
training loss: 0.1719681349954828
validation loss: 4.952070858713665
epoch: 1366
training loss: 0.17943367066756954
validation loss: 5.296327040113943
epoch: 1367
training loss: 0.09288393307300662
validation loss: 4.9672054213403545
epoch: 1368
training loss: 0.20793013798429988
validation loss: 4.935258999351009
epoch: 1369
training loss: 0.1812126527594314
validation loss: 5.2945343677599235
epoch: 1370
training loss: 0.09868768575847621
validation loss: 4.971998104787605
epoch: 1371
training loss: 0.24428879281822596
validation loss: 4.922218683547529
epoch: 1372
training loss: 0.18374251669043667
validation loss: 5.2900443340103935
epoch: 1373
training loss: 0.10717444831044859
validation loss: 4.97124867412996
epoch: 1374
training loss: 0.27909714840299626
validation loss: 4.912126152680788
epoch: 1375
training loss: 0.19127329694443088
validation loss: 5.2911021195768075
epoch: 1376
training loss: 0.1143071571856185
validation loss: 4.958501448547024
epoch: 1377
trai

epoch: 1470
training loss: 0.07163042650551349
validation loss: 4.985733550950584
epoch: 1471
training loss: 0.08207281881413804
validation loss: 4.959144953948324
epoch: 1472
training loss: 0.11096537762284865
validation loss: 4.828595702010807
epoch: 1473
training loss: 0.07456881433148886
validation loss: 4.998862018890866
epoch: 1474
training loss: 0.07948407084062203
validation loss: 4.946418485343417
epoch: 1475
training loss: 0.11486025617198597
validation loss: 4.8275994427050435
epoch: 1476
training loss: 0.0784614355427824
validation loss: 5.012594221880318
epoch: 1477
training loss: 0.07690637808503306
validation loss: 4.932724185455358
epoch: 1478
training loss: 0.1187217853152987
validation loss: 4.826365098089769
epoch: 1479
training loss: 0.08347925530249009
validation loss: 5.027276685066137
epoch: 1480
training loss: 0.07435778066952381
validation loss: 4.917727182406724
epoch: 1481
training loss: 0.12243513190283635
validation loss: 4.824972958056059
epoch: 1482
train

## Mean Absolute Error of Housing Predictions

In [ ]:
mae = mean_absolute_error(y, nn.predict(X_std))
print('Mean absolute error: $%0.2f'%(mae*1000))

Compare these to results to those in nn_tuning_example.ipynb.  Goal: Get MAE Under $1000 with our NN.  Then, we know our NN is working well and can use it on the dataset for this project.

## Compare to Linear Regression

In [8]:
class LR:
    
    def fit(self, X_train, y_train):
        # create vector of ones...
        ones = np.ones(shape=len(X_train))[..., None]
        #...and add to feature matrix
        X = np.concatenate((ones, X_train), 1)
        #calculate coefficients using closed-form solution
        self.coeffs = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y_train)
        
    def predict(self, X_test):
        ones = np.ones(shape=len(X_test))[..., None]
        X_test = np.concatenate((ones, X_test), 1)
        y_hat = X_test.dot(self.coeffs)
        return y_hat

## Linear Regression MAE

In [9]:
lr = LR()
lr.fit(X, y)
mae = mean_absolute_error(y, lr.predict(X_std))
print('Mean absolute error: $%0.2f'%(mae*1000))

Mean absolute error: $17885.89
